### Used Libraries<a class="anchor" id="chapter1"></a>

In [16]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

### Access to the DB <a class="anchor" id="chapter2"></a>

In [17]:
db_config = {'user': 'practicum_student',         # username
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # connection port
             'db': 'data-analyst-eth-payouts-db'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})


In [18]:
engine.table_names()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  """Entry point for launching an IPython kernel.


['payout', 'plan']

#  Table Queries <a class="anchor" id="chapter3"></a>

function that takes a query and return dataframe for general use

In [19]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

# **1. How many users got paid?**


In [20]:
query = '''
        SELECT 
          COUNT(DISTINCT payout.user_id) as users_paid
        FROM
          payout
        '''
queryResult(query)

,users_paid
0,433


# **2. Show the 5 users with the highest payouts**



In [21]:
query = '''
        SELECT 
          DISTINCT payout.user_id,
          SUM(payout.payout) as payout_sum
        FROM
          payout
        GROUP BY
          payout.user_id
        ORDER BY
          payout_sum DESC
        LIMIT 5;
        '''
queryResult(query)

,user_id,payout_sum
0,1537,10.74169
1,3051,9.73121
2,1512,8.80816
3,1127,8.78861
4,4848,8.42445


# **3. Show the 5 users with the lowest payouts**

In [22]:
query = '''
        SELECT 
          DISTINCT payout.user_id,
          SUM(payout.payout) as payout_sum
        FROM
          payout
        GROUP BY
          payout.user_id
        ORDER BY
          payout_sum
        LIMIT 5;
        '''
queryResult(query)

,user_id,payout_sum
0,2003,0.00775
1,3410,0.00818
2,2813,0.02914
3,2462,0.03381
4,4467,0.05056


# **4. How much ether was paid out in November 2020?**

In [23]:
query = '''
        SELECT 
          SUM(payout.payout) as payout_sum
        FROM
          payout
        WHERE
          payout.date :: date between '2020-11-1' and '2020-11-30'
        '''
queryResult(query)

,payout_sum
0,166.0118


# **5. Which plan is the most popular?**

In [24]:
query = '''
        SELECT
          plan."Plan",
          COUNT(plan."Plan")
        FROM
          plan
        GROUP BY
          plan."Plan"
        ORDER BY
          COUNT(plan."Plan") DESC
        '''
queryResult(query)

,Plan,count
0,Free,220
1,Premium,213


# **6. Which plan is the most popular amongst Linux users?**

In [25]:
query = '''
        SELECT
          plan."Plan",
          COUNT(plan."Plan") 
        FROM
          plan
        WHERE plan."OS" LIKE '%Linux%'
        GROUP BY
          plan."Plan"
        ORDER BY
          COUNT(plan."Plan") DESC
        '''
queryResult(query)

,Plan,count
0,Premium,76
1,Free,68


# **7. What is the percentage of payout between the different plans?**

In [26]:
query = '''
      SELECT
        DISTINCT plan."Plan",
        SUM(payout.payout) OVER (PARTITION BY plan."Plan") / (SELECT 
          SUM(payout.payout) / 100 as payout_sum
        FROM
          payout) as ratio

      FROM
        payout LEFT JOIN plan ON payout.user_id = plan.user_id
      GROUP BY
        plan."Plan",
        payout.payout
        '''
         
queryResult(query)

,Plan,ratio
0,Free,53.738342
1,Premium,46.028049


# **8. Users of which operating system earned more in payouts?**

In [31]:
query = '''
      SELECT
        DISTINCT plan."OS",
        SUM(payout.payout) as payout_os

      FROM
        payout LEFT JOIN plan ON payout.user_id = plan.user_id
      GROUP BY
        plan."OS"
      ORDER BY
        payout_os DESC
        '''
        
queryResult(query)

,OS,payout_os
0,Linux,322.00353
1,Windows,316.03701
2,MAC,267.41123


# **9. What is the average payout amount per user for each of the OS in July 2020?**

In [36]:
query = '''
      SELECT
        DISTINCT plan."OS",
        AVG(payout.payout) avg_payout_os

      FROM
        payout LEFT JOIN plan ON payout.user_id = plan.user_id
      WHERE
       payout.date :: date between '2020-07-1' and '2020-07-31'
      GROUP BY
        plan."OS"
      ORDER BY
        plan."OS"
      
        '''
        
queryResult(query)

,OS,avg_payout_os
0,Linux,1.931114
1,MAC,1.640537
2,Windows,1.590600


# **10. What is the daily share of ether earned by users from Linux that are in the free plan in this data?**

In [38]:
query = '''
      SELECT
        DISTINCT payout.date :: date,
        payout.user_id,
        payout.payout * 100 / SUM(payout.payout) OVER (PARTITION BY payout.date :: date) as ratio
       
      FROM
        payout LEFT JOIN plan ON payout.user_id = plan.user_id
      WHERE
        plan."OS" LIKE '%Linux%' and  plan."Plan" LIKE '%Free%'
      ORDER BY
        payout.date :: date
        '''
queryResult(query)

,date,user_id,ratio
0,2020-07-07,3415,100.000000
1,2020-07-08,3146,100.000000
2,2020-07-11,2154,100.000000
3,2020-07-13,4235,100.000000
4,2020-07-14,3542,100.000000
...,...,...,...
94,2021-01-02,2935,100.000000
95,2021-01-03,1330,100.000000
96,2021-01-04,2128,100.000000
97,2021-01-05,4131,63.505578
